# Multi-variate time series classification with sktime-dl
[Github](https://github.com/sktime/sktime-dl)

In this notebook, we use sktime-dl to perform for multi-variate time series classification by deep learning.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import GridSearchCV
from sktime.datasets import load_basic_motions
from sktime_dl.classification import CNNClassifier

sns.set_style('whitegrid') 

2022-11-06 15:53:21.198591: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Load a dataset
The Basic Motions dataset, from [timeseriesclassification.com](http://www.timeseriesclassification.com), has time series in six dimensions.

In [2]:
X_train, y_train = load_basic_motions(split='train', return_X_y=True)
X_test, y_test = load_basic_motions(split='test', return_X_y=True)
X_train.head()

,dim_0,dim_1,dim_2,dim_3,dim_4,dim_5
0,0 0.079106 1 0.079106 2 -0.903497 3...,0 0.394032 1 0.394032 2 -3.666397 3...,0 0.551444 1 0.551444 2 -0.282844 3...,0 0.351565 1 0.351565 2 -0.095881 3...,0 0.023970 1 0.023970 2 -0.319605 3...,0 0.633883 1 0.633883 2 0.972131 3...
1,0 0.377751 1 0.377751 2 2.952965 3...,0 -0.610850 1 -0.610850 2 0.970717 3...,0 -0.147376 1 -0.147376 2 -5.962515 3...,0 -0.103872 1 -0.103872 2 -7.593275 3...,0 -0.109198 1 -0.109198 2 -0.697804 3...,0 -0.037287 1 -0.037287 2 -2.865789 3...
2,0 -0.813905 1 -0.813905 2 -0.424628 3...,0 0.825666 1 0.825666 2 -1.305033 3...,0 0.032712 1 0.032712 2 0.826170 3...,0 0.021307 1 0.021307 2 -0.372872 3...,0 0.122515 1 0.122515 2 -0.045277 3...,0 0.775041 1 0.775041 2 0.383526 3...
3,0 0.289855 1 0.289855 2 -0.669185 3...,0 0.284130 1 0.284130 2 -0.210466 3...,0 0.213680 1 0.213680 2 0.252267 3...,0 -0.314278 1 -0.314278 2 0.018644 3...,0 0.074574 1 0.074574 2 0.007990 3...,0 -0.079901 1 -0.079901 2 0.237040 3...
4,0 -0.123238 1 -0.123238 2 -0.249547 3...,0 0.379341 1 0.379341 2 0.541501 3...,0 -0.286006 1 -0.286006 2 0.208420 3...,0 -0.098545 1 -0.098545 2 -0.023970 3...,0 0.058594 1 0.058594 2 0.175783 3...,0 -0.074574 1 -0.074574 2 0.114525 3...


In [3]:
# The class labels
np.unique(y_train)

array(['badminton', 'running', 'standing', 'walking'], dtype='<U9')

# Train a deep neural network classifier
Here we choose to use the CNN (convolutional neural network) classifier. Other classifiers provided by sktime-dl include MLP, ResNet, InceptionTime and MCDCNN (Multi Channel Deep Convolutional Neural Network)

In [4]:
network = CNNClassifier(nb_epochs=200, verbose=False)
network.fit(X_train, y_train)
network.score(X_test, y_test)

ValueError: Data seen by CNNClassifier instance has multivariate series, but this CNNClassifier instance cannot handle multivariate series. Calls with multivariate series may result in error or unreliable results.

# Grid Search
sktime-dl is compatible with scikit-learn and can use sklearn's GridSearchCV.

Here we search over two parameters, CNN kernel size and pooling size.

In [16]:
param_grid = {'kernel_size': [7, 9],
              'avg_pool_size': [3, 5]}
grid = GridSearchCV(network, param_grid=param_grid, cv=5)   
grid.fit(X_train, y_train)
    
print("Best cross-validation accuracy: {:.2f}".format(grid.best_score_))
print("Test set score: {:.2f}".format(grid.score(X_test, y_test)))
print("Best parameters: {}".format(grid.best_params_))

Best cross-validation accuracy: 0.30
Test set score: 0.95
Best parameters: {'avg_pool_size': 3, 'kernel_size': 7}
